# Giai đoạn 1

***Y1-01***
- ✓ Nạp dữ liệu từ csv, kiểm tra thông tin cơ bản.
- ✓ Hiển thị một số dữ liệu ở đầu và cuối.
- ✓ Xác định cột không cần thiết, thực hiện loại bỏ để làm sạch.
- ✓ Chuyển đổi kiểu dữ liệu ở các cột cho phù hợp (ví dụ cột chứa dữ liệu ngày tháng)


In [1]:
import pandas as pd

In [2]:
# Đường dẫn file 
file=r"E:\FPT\DAT2051\Assignment\covid_19_data.csv"
# Tải dữ liệu
df=pd.read_csv(file)

In [3]:
# Chuyển đổi dữ liệu của 2 cột date
df['ObservationDate']=df['ObservationDate'].astype('datetime64[s]')
df['Last Update']=df['Last Update'].astype('datetime64[s]')
# xóa cột SnoSno
df.drop(columns=['SNo'],inplace=True)
# Thay thế tỉnh nan thành UnknownUnknown
df.loc[df['Province/State'].isna(),'Province/State']='Unknown'

In [4]:
# Xử lý giá trị âm 
df.loc[df['Confirmed']<0,'Confirmed'] = df.loc[df['Confirmed']<0,'Confirmed'] * -1 
df.loc[df['Deaths']<0,'Deaths'] = df.loc[df['Deaths']<0,'Deaths'] * -1 
df.loc[df['Recovered']<0,'Recovered'] = df.loc[df['Recovered']<0,'Recovered'] * -1 

In [5]:
# Xử lý giá trị tích lũy
list_temp=['Confirmed','Deaths','Recovered']
for _ in list_temp:
    col_name= _ +'Non_Cumlative'
    df[col_name]=(
    df.sort_values(['Country/Region','Province/State','ObservationDate'])
    .groupby(['Country/Region','Province/State'])[_]
    .diff()).fillna(df[_]
                    )

***Y1-02***
- ✓ Tính tổng số ca nhiễm, tử vong, và khỏi bệnh trên toàn thế giới.
- ✓ Tính tỷ lệ tử vong trên số ca nhiễm
- ✓ Lọc dữ liệu cho một quốc gia cụ thể
- ✓ Phân loại các quốc gia thành các nhóm dựa trên số lượng ca nhiễm

In [6]:
# Tính tổng ca nhiễm , ca tử vong, ca hồi phục.
total_case=df['ConfirmedNon_Cumlative'].sum()
total_dead=df['DeathsNon_Cumlative'].sum()
total_recovered=df['RecoveredNon_Cumlative'].sum()

In [7]:
# Tỷ lệ ca tử vong trên ca nhiễmnhiễm
print(round(total_dead/total_case,2)*100,"%")

2.0 %


In [8]:
# Lọc ra các quốc gia 
list_country=['france']
df[df['Country/Region'].str.lower().str.strip(' ').isin(list_country)]

,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,ConfirmedNon_Cumlative,DeathsNon_Cumlative,RecoveredNon_Cumlative
128,2020-01-24,Unknown,France,2020-01-24 17:00:00,2.0,0.0,0.0,2.0,0.0,0.0
171,2020-01-25,Unknown,France,2020-01-25 17:00:00,3.0,0.0,0.0,1.0,0.0,0.0
219,2020-01-26,Unknown,France,2020-01-26 16:00:00,3.0,0.0,0.0,0.0,0.0,0.0
268,2020-01-27,Unknown,France,2020-01-27 23:59:00,3.0,0.0,0.0,0.0,0.0,0.0
321,2020-01-28,Unknown,France,2020-01-28 23:00:00,4.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
306263,2021-05-29,Reunion,France,2021-05-30 04:20:55,24901.0,189.0,22796.0,0.0,0.0,0.0
306279,2021-05-29,Saint Barthelemy,France,2021-05-30 04:20:55,1005.0,1.0,462.0,0.0,0.0,0.0
306282,2021-05-29,Saint Pierre and Miquelon,France,2021-05-30 04:20:55,25.0,0.0,25.0,0.0,0.0,0.0
306320,2021-05-29,St Martin,France,2021-05-30 04:20:55,1915.0,12.0,1399.0,0.0,0.0,0.0


In [9]:
# Phân loại tình trạng dựa trên số ca nhiễmnhiễm
def country_segment(row):
    if row < 1000000:
        return 'Low'
    elif row < 2000000:
        return 'Medium'
    else:
        return 'High'
total_case_df=df.groupby('Country/Region')['ConfirmedNon_Cumlative'].sum().reset_index()
total_case_df['Segment']=total_case_df['ConfirmedNon_Cumlative'].apply(country_segment)

# GIAI ĐOẠN 2
***Y2-01***
- ✓ Kết hợp dữ liệu số ca nhiễm COVID-19 với dữ liệu dân số của các quốc gia để
- tính tỷ lệ nhiễm COVID-19 trên mỗi 100,000 dân.
- ✓ Tính số ca nhiễm mới hàng ngày trên toàn thế giới từ dữ liệu COVID-19.
- ✓ Tính số ca nhiễm COVID-19 hàng ngày trung bình trong một tuần để đánh giá sự biến động hàng tuần của dịch bệnh.


In [10]:
df_Y2=total_case_df

In [ ]:
# Tải và tải file lên
file_population=r"C:\Users\Nguyen Tuan Khoa\Downloads\population-unwpp.csv"
df_population=pd.read_csv(file_population)
# Đổi tên cột
df_population.rename(
    columns={'Entity':'Country/Region',
            'Population - Sex: all - Age: all - Variant: estimates':'Population'
            }
            ,inplace=True)
# Lấy trung bình dân số của năm 2020 và 2021
df_population=df_population[df_population['Year'].isin([2020,2021])].groupby(['Country/Region'])['Population'].mean().reset_index()

In [ ]:
# Merge dữ liệu theo phương thức left join.
# Key là Country/Region
df_population_covid=pd.merge(
    left=df_Y2,
    right=df_population,
    on='Country/Region',
    how='left')
df_population_covid

,Country/Region,ConfirmedNon_Cumlative,Segment,Population
0,Azerbaijan,1.0,Low,NaN
1,"('St. Martin',)",2.0,Low,NaN
2,Afghanistan,70111.0,Low,39534694.5
3,Albania,132297.0,Low,2860797.0
4,Algeria,128456.0,Low,44401597.0
...,...,...,...,...
224,West Bank and Gaza,307838.0,Low,NaN
225,Yemen,6731.0,Low,36637550.5
226,Zambia,94751.0,Low,19331502.5
227,Zimbabwe,38933.0,Low,15662054.0
